In [77]:
import pandas as pd

# We can use the read_excel method to read data from an Excel file
dataframe = pd.read_excel('files/P6-SuperStoreUS-2015.xlsx')

# Let's check out the first 5 rows from our excel file
dataframe.head()

,Row ID,Order Priority,Discount,Unit Price,Shipping Cost,Customer ID,Customer Name,Ship Mode,Customer Segment,Product Category,...,Region,State or Province,City,Postal Code,Order Date,Ship Date,Profit,Quantity ordered new,Sales,Order ID
0,20847,High,0.01,2.84,0.93,3,Bonnie Potter,Express Air,Corporate,Office Supplies,...,West,Washington,Anacortes,98221,2015-01-07,2015-01-08,4.5600,4,13.01,88522
1,20228,Not Specified,0.02,500.98,26.00,5,Ronnie Proctor,Delivery Truck,Home Office,Furniture,...,West,California,San Gabriel,91776,2015-06-13,2015-06-15,4390.3665,12,6362.85,90193
2,21776,Critical,0.06,9.48,7.29,11,Marcus Dunlap,Regular Air,Home Office,Furniture,...,East,New Jersey,Roselle,7203,2015-02-15,2015-02-17,-53.8096,22,211.15,90192
3,24844,Medium,0.09,78.69,19.99,14,Gwendolyn F Tyson,Regular Air,Small Business,Furniture,...,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-14,803.4705,16,1164.45,86838
4,24846,Medium,0.08,3.28,2.31,14,Gwendolyn F Tyson,Regular Air,Small Business,Office Supplies,...,Central,Minnesota,Prior Lake,55372,2015-05-12,2015-05-13,-24.0300,7,22.23,86838


In [78]:
# We can check out the length of the dataframe
len(dataframe)

1952

In [79]:
import sqlite3

# Create a connection to a local database (This will create a file called mydatabase.db)
connection = sqlite3.connect('mydatabase.db')

# Copy the dataframe to our SQLite database
dataframe.to_sql('mytable', connection, if_exists='replace')

1952

In [80]:
import os
from groq import Groq
import dotenv

# Load API key from the .env file
dotenv.load_dotenv()

# Fetch the API key
api_key = os.getenv('GROQ_API_KEY')

# Ensure the API key was loaded properly
if api_key is None:
    raise ValueError("API key not found! Check your .env file and environment variables.")

# Initialize the Groq client with the API key
client = Groq(api_key=api_key)

# Example chat completion request to Groq
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": "are the universals real and distinct entities or are they mental constructs?",
        }
    ],
    model="llama3-8b-8192",  # Specify the model name supported by Groq
)

# Output the result
print(chat_completion.choices[0].message.content)



What a fascinating question!

The concept of universals has been debated by philosophers, linguists, and cognitive scientists for centuries. The question is whether universals, such as numbers, properties, or concepts, exist independently of human minds or are simply mental constructs created by our perceptions and cognition.

There are different perspectives on this issue, and I'll provide an overview of some arguments for both sides:

**Real and distinct entities:**

1. **Platonism**: In Plato's philosophy, universals are eternal, abstract objects that exist independently of the physical world. He believed that these Forms or Ideas are the ultimate reality, and the physical world is just a shadow or imitation of this realm of abstract Forms.
2. **Realism**: Some philosophers argue that universals have a mind-independent existence, similar to physical objects. They believe that properties, such as redness or tallness, are inherent in objects and exist regardless of human perception.
3

In [82]:
import google.generativeai as genai
import dotenv
import os

# Load API key from the .env file
dotenv.load_dotenv()

# Fetch the API key
api_key = os.getenv('GEMINI_API_KEY')

# Ensure the API key was loaded properly
if api_key is None:
    raise ValueError("API key not found! Check your .env file and environment variables.")

# Configure the Gemini client with the API key
genai.configure(api_key=api_key)

# Initialize the Generative Model
gemini = genai.GenerativeModel("gemini-1.5-flash")


In [72]:
# Test out the model by generating some text
text = gemini.generate_content("are universals real and distinct entities or merely mental constructs?").text
print(text)

The question of whether universals are real and distinct entities or merely mental constructs is a fundamental philosophical debate that has been ongoing for centuries. There are two main opposing views:

**Realism:**

* **Universals are real, distinct entities:** Realists believe that universals exist independently of our minds and are not simply mental constructs. They argue that universals are the essence of things, the common features that make them what they are. 
* **Examples:** The concept of "redness" exists independently of any red objects, and all red objects share this same universal property. 
* **Arguments for realism:**
    * **The problem of universals:** How can we explain the fact that different objects can be classified as belonging to the same category, like "redness"? This suggests there is a shared underlying reality.
    * **Predictability:** If universals were merely mental constructs, we wouldn't be able to predict future occurrences with any accuracy.

**Nomina

In [83]:
# The doc string is important, as it is passed to the LLM as a description of the function
def sql_query(query: str):
    """Run a SQL SELECT query on a SQLite database and return the results."""
    return pd.read_sql_query(query, connection).to_dict(orient='records')

In [85]:
system_prompt = """
You are an expert SQL analyst. When appropriate, generate SQL queries based on the user question and the database schema.
When you generate a query, use the 'sql_query' function to execute the query on the database and get the results.
Then, use the results to answer the user's question.

database_schema: [
    {
        table: 'mytable',
        columns: [
            {
                name: 'Row ID', l
                type: 'int'
            },
            {
                name: 'Order Priority',
                type: 'string'
            },
            {
                name: 'Discount',
                type: 'float'
            },
            {
                name: 'Unit Price',
                type: 'float'
            },
            {
                name: 'Shipping Cost',
                type: 'float'
            },
            {
                name: 'Customer ID',
                type: 'int'
            },
            {
                name: 'Customer Name',
                type: 'string'
            },
            {
                name: 'Ship Mode',
                type: 'string'
            },
            {
                name: 'Customer Segment',
                type: 'string'
            },
            {
                name: 'Product Category',
                type: 'string'
            },
            {
                name: 'Product Sub-Category',
                type: 'string'
            },
            {
                name: 'Product Container',
                type: 'string'
            },
            {
                name: 'Product Name',
                type: 'string'
            },
            {
                name: 'Product Base Margin',
                type: 'float'
            },
            {
                name: 'Country',
                type: 'string'
            },
            {
                name: 'Region',
                type: 'string'
            },
            {
                name: 'State or Province',
                type: 'string'
            },
            {
                name: 'City',
                type: 'string'
            },
            {
                name: 'Postal Code',
                type: 'string'
            },
            {
                name: 'Order Date',
                type: 'datetime'
            },
            {
                name: 'Ship Date',
                type: 'datetime'
            },
            {
                name: 'Profit',
                type: 'float'
            },
            {   
                name: 'Quantity ordered new',
                type: 'int' 
            },
            {
                name: 'Sales',
                type: 'float'
            },
            {
                name: 'Order ID',
                type: 'int'   
            }           
        ]
    }
]
""".strip() # Call strip to remove leading/trailing whitespace

In [86]:
# Let's create a new Gemini instance with the SQL query function and the system prompt
sql_gemini = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=[sql_query], system_instruction=system_prompt)

In [87]:
# We begin our chat with Gemini and allow it to use tools when needed
chat = sql_gemini.start_chat(enable_automatic_function_calling=True)

# Let's ask our first question
chat.send_message("which mode of shipping is most used").text

'The most used shipping mode is Regular Air. \n'

In [88]:
chat.send_message("which product category has the highest unit price?").text

'The product category with the highest unit price is Technology. \n'

In [89]:
chat.send_message("which region bring in the most profit?").text

'The region that brings in the most profit is East. \n'